<a href="https://colab.research.google.com/github/derkmed/COMSE6998_025_Project/blob/main/get_gpt_instruction_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dataclasses import dataclass
from google.auth import default
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from openai import OpenAI
import os
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from tqdm import tqdm

ModuleNotFoundError: ignored

In [ ]:
!pip install -qU langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
IN_SHEET_URL = "https://docs.google.com/spreadsheets/d/1ZlzC-n4TJuwHwzRBL9hjZx8ZU7jnI-5Nzvjw3xo3l5E/edit#gid=2126251830" #@param {type: "string"}
IN_SHEET_NAME = "Rater Assigment" #@param {type: "string"}
#@markdown ---
OUT_SHEET_URL = "https://docs.google.com/spreadsheets/d/1kzQccQaoyFEV3_6lFZ3jBPZNBpMOuYZGeDrx1AYxQpw/edit#gid=0" #@param {type: "string"}

#@markdown ---
OPENAI_API_KEY="" #@param {type: "string"}
OPENAI_ORG_ID="" #@param {type: "string"}



os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_ORG_ID"] = OPENAI_ORG_ID

NameError: ignored

In [ ]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()


#@title Set up Sheets Authentication
creds, _ = default()
gc = gspread.authorize(creds)

NameError: ignored

In [ ]:
in_sheet = gc.open_by_url(IN_SHEET_URL)
examples_sheet_data = in_sheet.worksheet(IN_SHEET_NAME).get_all_values()
ratings_df = pd.DataFrame(examples_sheet_data[1:], columns=examples_sheet_data[0])

NameError: ignored

In [ ]:
ratings_df

,w_id,video_type,video_title,Instruction Family,Video,Instruction,Assignee,Is the instruction related to the Instruction Family? (Fill this In),Caption (Fill this In),
0,2,leg extension,leg extension_22.mp4,Performance,https://drive.google.com/file/d/1jzWbO1kK3Ienx...,\r\nHow does leg extensions contribute to the ...,Vineet Bhardwaj,TRUE,"While seated, the person extends their legs ag...",
1,3,leg extension,leg extension_2.MOV,Time,https://drive.google.com/file/d/1thwKuzDsDatee...,Are the reps slows or fast?,Vineet Bhardwaj,TRUE,The person is sitting on a leg extension machi...,
2,4,pull Up,pull up_16.mp4,Difficulty,https://drive.google.com/file/d/1kHlsWuCo9Dkg8...,Can a beginner do 20 repetitions of this exerc...,Vineet Bhardwaj,FALSE,Bodyweight pull up is performed by a person gr...,
3,5,lateral raise,lateral raise_33.mp4,Accesibility,https://drive.google.com/file/d/101Y1KAZgeIOT0...,Can this be done in a wheelchair?,Vineet Bhardwaj,TRUE,Person is doing lateral raises by standing wit...,
4,6,hammer curl,hammer curl_4.MOV,Accesibility,https://drive.google.com/file/d/14CTC6d9Sd_kHn...,Can this exercise be modified for someone with...,Vineet Bhardwaj,FALSE,"Individuals stands with dumbbells in hand, arm...",
...,...,...,...,...,...,...,...,...,...,...
477,479,leg extension,leg extension_15.mp4,Impact,https://drive.google.com/file/d/1vfV9u8tH6kqaj...,What specific muscles are benefitted by perfor...,Yunzikai Chen,TRUE,"There's an individual sitting on the bench, gr...",
478,480,lat pulldown,lat pulldown_9.mp4,Impact,https://drive.google.com/file/d/1KTjcDhFs5H9Do...,What will happen to the user's feet muscles fr...,Yunzikai Chen,TRUE,"There's an individual sitting on the bench, fa...",
479,481,leg extension,leg extension_10.mp4,Alternatives,https://drive.google.com/file/d/17z8Q5M-w6eHOQ...,Why can't an individual complete this exercise...,Yunzikai Chen,TRUE,"There's an individual sitting on the bench, le...",
480,482,bench press,bench press_25.mp4,Sports,https://drive.google.com/file/d/1AQoBACzd9hUoO...,Will a tennis player benefit from this \nexcer...,Yunzikai Chen,TRUE,"There's an individual lying on a bench, knees ...",


In [ ]:
CAPTION = "The person is sitting on a leg extension machine, securing his ankles under the padded lever. By extending his legs at a swift pace against resistance, the quadriceps muscles are targeted, providing strength & muscle development in the front of the thighs." #@param {type: "string"}
INSTRUCTION = "Are the reps slows or fast?" #@param {type: "string"}
MODEL_NAME = "gpt-4" #@param {type: "string"}

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate


# Define the OpenAI model.
# Please refer to https://github.com/hwchase17/langchain/blob/master/langchain/llms/openai.py
openai_llm = ChatOpenAI(
    model_name=MODEL_NAME,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.0,
    max_tokens=128
)

prompt_template = """
I am a customized version of ChatGPT that serves as a video assistant, optimized
specifically to assist users by fulfilling their instructions and
answering their questions. When details are missing from the user's instruction,
I will make educated guesses to provide useful responses but will also note
when additional information might be needed for a more accurate answer.

Consider a video captured with this description: "{caption}". I will not
mention that this is only a description; I will pretend that I have watched the video.
I will now briefly follow the provided instruction and add a short explanation:
Instruction: {instruction}
Response:"""

zero_shot_prompt = PromptTemplate(
    input_variables=["caption", "instruction"],
    template=prompt_template
)

chain = LLMChain(llm=openai_llm, prompt=zero_shot_prompt)
response = chain.run(caption=CAPTION, instruction=INSTRUCTION)
def get_gpt_response(caption: str, instruction: str, is_relevant: bool) -> str:
  chain = LLMChain(llm=openai_llm, prompt=zero_shot_prompt)
  if is_relevant:
    return chain.run(caption=caption, instruction=instruction)
  return ""

print(f"########## DEMO ##########\n\nCAPTION: {CAPTION}\n\nINSTRUCTION:{INSTRUCTION}\n\nRESPONSE:{get_gpt_response(CAPTION, INSTRUCTION, True)}")

########## DEMO ##########

CAPTION: The person is sitting on a leg extension machine, securing his ankles under the padded lever. By extending his legs at a swift pace against resistance, the quadriceps muscles are targeted, providing strength & muscle development in the front of the thighs.

INSTRUCTION:Are the reps slows or fast?

RESPONSE:The reps are performed at a swift pace, indicating they are fast. This speed can help increase the intensity of the workout and potentially lead to greater muscle development.


In [ ]:
prep_gpt_df = ratings_df.copy()

In [ ]:
# #@title GPT 3.5 Turbo x LangChain
# langchain.debug = False
# gpt_data = {
#     'w_id': [],
#     'inst_family': [],
#     'caption': [],
#     'instruction': [],
#     'is_relevant': [],
#     'gpt_response': [],
# }
# t = tqdm(total=len(prep_gpt_df))
# for index, row in prep_gpt_df.iterrows():
#   wid = row['w_id']
#   inst_family = row['Instruction Family']
#   caption = row['Caption (Fill this In)']
#   instruction = row['Instruction']
#   is_relevant = row['Is the instruction related to the Instruction Family? (Fill this In)']
#   response = get_gpt_response(caption, instruction, is_relevant)

#   gpt_data['w_id'].append(wid)
#   gpt_data['inst_family'].append(inst_family)
#   gpt_data['caption'].append(caption)
#   gpt_data['instruction'].append(instruction)
#   gpt_data['is_relevant'].append(is_relevant)
#   gpt_data['gpt_response'].append(response)
#   t.update(1)

  1%|          | 3/482 [00:05<15:17,  1.92s/it]

KeyboardInterrupt: ignored

In [ ]:
#@title GPT 4 with Exponential Backoff
gpt_data = {
    'w_id': [],
    'inst_family': [],
    'caption': [],
    'instruction': [],
    'is_relevant': [],
    'gpt_response': [],
}


# client = OpenAI()

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_gpt_response(caption: str, instruction: str, is_relevant: bool) -> str:
  chain = LLMChain(llm=openai_llm, prompt=zero_shot_prompt)
  if is_relevant:
    return chain.run(caption=caption, instruction=instruction)
  return ""
# def completion_with_backoff(**kwargs):
#     return client.completions.create(**kwargs)


t = tqdm(total=len(prep_gpt_df))
j = 0
for index, row in prep_gpt_df.iterrows():
  wid = row['w_id']
  inst_family = row['Instruction Family']
  caption = row['Caption (Fill this In)']
  instruction = row['Instruction']
  is_relevant = row['Is the instruction related to the Instruction Family? (Fill this In)']


  prompt = prompt_template.format(caption=caption, instruction=instruction)
  response = get_gpt_response(caption, instruction, is_relevant)
  print(response)


  gpt_data['w_id'].append(wid)
  gpt_data['inst_family'].append(inst_family)
  gpt_data['caption'].append(caption)
  gpt_data['instruction'].append(instruction)
  gpt_data['is_relevant'].append(is_relevant)
  gpt_data['gpt_response'].append(response)
  t.update(1)
  if index % 10 == 0:
    out_sheet = gc.open_by_url(OUT_SHEET_URL)
    out_wksht = out_sheet.worksheet(f"Sheet{j}")
    gpt_df = pd.DataFrame.from_dict(gpt_data)
    set_with_dataframe(out_wksht, gpt_df)
    j += 1

  0%|          | 1/482 [00:06<55:27,  6.92s/it]

Leg extensions are a highly effective exercise for strengthening the leg muscles, particularly the quadriceps. The quadriceps are a group of four muscles located at the front of the thigh. When you perform a leg extension, you're isolating these muscles and making them work against resistance. This resistance can be adjusted to match your strength level, allowing you to progressively overload the muscles, which is key for muscle growth and strength gains. Additionally, leg extensions can help improve muscular endurance in the lower body, which can enhance performance in various sports and daily activities. However, it's important to note that while leg extensions can be a beneficial part of a comprehensive


  0%|          | 2/482 [00:10<41:11,  5.15s/it]

The reps are performed at a swift pace, indicating they are fast. This speed can help increase the intensity of the workout and potentially lead to greater muscle development.


  1%|          | 3/482 [00:14<35:57,  4.50s/it]

...

The seat or bench in this exercise provides stability and support, allowing the individual to focus on isolating and working the quadriceps muscles. Without a seat, it would be challenging to maintain the correct posture and balance, which could lead to ineffective exercise or potential injury. The seat also helps in ensuring that the force exerted is primarily from the quadriceps, making the exercise more effective.


100%|█████████▉| 481/482 [51:28<00:04,  4.43s/it]

Yes, a tennis player could benefit from this exercise. The movement described is a bench press, which is a compound exercise that works several muscles at once, including the chest, shoulders, and triceps. These muscles are used in tennis for actions like serving, volleying, and hitting groundstrokes. Strengthening these muscles could potentially improve a tennis player's power and endurance on the court. However, it's important to note that sport-specific training should also be incorporated for optimal performance.


100%|██████████| 482/482 [51:35<00:00,  5.23s/it]

Yes, practicing this exercise, which is a bench press, can potentially make push-ups easier over time. Both exercises work similar muscle groups, including the chest, shoulders, and triceps. By strengthening these muscles through bench presses, an individual could improve their push-up performance. However, it's important to note that push-ups also engage other muscles like the core and lower body, which are not as heavily targeted in a bench press. Therefore, a balanced workout routine would be most beneficial.


In [ ]:
out_sheet = gc.open_by_url(OUT_SHEET_URL)
out_wksht = out_sheet.worksheet(f"{MODEL_NAME}-inferences-demo")
set_with_dataframe(out_wksht, gpt_df)

In [ ]:
gpt_df

NameError: ignored

In [ ]:
in_sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1INKXpMuiltEm-yL9Ud-dQOVJgbFxAFafT9VGQJ_mLRg/edit#gid=1467046577")
examples_sheet_data = in_sheet.worksheet("gpt-4-inferences").get_all_values()
vals_df = pd.DataFrame(examples_sheet_data[1:], columns=examples_sheet_data[0])

NameError: ignored